In [1]:
from datasets import keepcontrol
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [46]:
# Global variables
PATH = "/mnt/neurogeriatrics_data/Keep Control/Data/lab dataset/rawdata"
DEMOGRAPHICS_FILE = "/mnt/neurogeriatrics_data/Keep Control/Data/lab dataset/rawdata/participants.tsv"
TRACKED_POINTS = ["left_ankle", "right_ankle"]
CLASSIFICATION_TASK = "phases"
WIN_LEN = 400

In [47]:
ds_train, ds_val, ds_test = keepcontrol.load_data(
    path=PATH,
    filename=DEMOGRAPHICS_FILE,
    tracked_points=TRACKED_POINTS,
    incl_magn=False,
    classification_task=CLASSIFICATION_TASK,
    win_len=WIN_LEN
)

Load dataset from pickle ...


In [48]:
# Split datasets in data and labels
(train_data, train_labels, train_filenames, train_ids) = ds_train
(val_data, val_labels, val_filenames, val_ids) = ds_val

In [49]:
train_labels.dtype

dtype('float64')

In [50]:
train_labels.shape

(3502, 400, 1)

In [51]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Dense
from tcn import TCN, tcn_full_summary

In [59]:
def make_model(input_shape):
    
    # Define the layers that make up the architecture
    input_layer = Input(shape=(None, input_shape[-1]), name="input_layer")
    tcn_layer = TCN(
        nb_filters=16,
        kernel_size=3,
        nb_stacks=1,
        dilations=(1, 2, 4, 8, 16, 32),
        padding="causal",
        return_sequences=True,
        use_skip_connections=True,
        use_batch_norm=True,
        name="tcn_layer"
    )(input_layer)
    output_layer = Dense(units=1, activation="sigmoid", name="output_layer")(tcn_layer)
    
    # Define the model
    tcn_model = Model(inputs=input_layer, outputs=output_layer, name="tcn_model")
    
    # Compile the model
    tcn_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    return tcn_model

In [60]:
tcn_model = make_model(train_data.shape[1:])
tcn_model.summary()

Model: "tcn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, None, 6)]         0         
                                                                 
 tcn_layer (TCN)             (None, None, 16)          9808      
                                                                 
 output_layer (Dense)        (None, None, 1)           17        
                                                                 
Total params: 9,825
Trainable params: 9,441
Non-trainable params: 384
_________________________________________________________________


In [61]:
history = tcn_model.fit(
    x=train_data,
    y=train_labels,
    epochs=5,
    batch_size=128,
    validation_data=(val_data, val_labels)
)

Epoch 1/5
28/28 [==============================] - 3s 28ms/step - loss: nan - binary_accuracy: 0.6789 - val_loss: nan - val_binary_accuracy: 0.6812
Epoch 2/5
28/28 [==============================] - 0s 15ms/step - loss: nan - binary_accuracy: 0.6789 - val_loss: nan - val_binary_accuracy: 0.6812
Epoch 3/5
28/28 [==============================] - 0s 15ms/step - loss: nan - binary_accuracy: 0.6789 - val_loss: nan - val_binary_accuracy: 0.6812
Epoch 4/5
28/28 [==============================] - 0s 15ms/step - loss: nan - binary_accuracy: 0.6789 - val_loss: nan - val_binary_accuracy: 0.6812
Epoch 5/5
28/28 [==============================] - 0s 15ms/step - loss: nan - binary_accuracy: 0.6789 - val_loss: nan - val_binary_accuracy: 0.6812
